In [1]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.catboost import CatBoost as Model

In [2]:
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [5]:
def_features = ['duels90','duelswonpct','succssdefact90','defduels90','defduelswpct','aerduels90','aerduelspct','sltackles90','padjsltackles',
                'shotblocks90','intrcpt90','padjintrcpt','fouls90','ycards90','rcards90','passes90','accpassespct','fwdpasses90','accfwdpassespct',
                'bpasses90','accbpassespct','latpasses90','acclatpassespct','shortmedpasses90','accshortmedpassespct','longpasses90','acclongpassesct',
                'avgpasslen','avglongpasslen']

In [6]:
model_scores = pd.DataFrame()
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = Model(data = td.data, features = features, full_feature_set=full_feature_set)
    display(m.data[m.features], features)
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    global model_scores
    model_scores = pd.concat([model_scores, m.score(thresh=10)])
    # display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))
    display(m.top_n_predictions(20, worst=True))
    m.calculate_effectiveness()

In [7]:
pipeline('DEF', def_features)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,bpasses90,accbpassespct,latpasses90,acclatpassespct,shortmedpasses90,accshortmedpassespct,longpasses90,acclongpassesct,avgpasslen,avglongpasslen
4,19,0.0,1,85.5,1816,1871,1429.238095,1561.400000,75.0,33,...,4.49,97.44,28.80,97.00,55.93,95.16,4.75,57.58,21.02,35.89
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,5.06,89.39,11.49,94.00,33.70,92.95,4.21,49.09,19.57,32.68
8,25,0.0,1,65.0,1871,2041,1561.400000,1696.800000,55.0,25,...,8.76,94.12,21.21,82.79,47.66,91.26,4.47,62.50,19.02,33.24
12,21,0.0,1,55.0,1747,1842,1696.800000,1696.800000,35.0,35,...,3.67,86.86,5.69,77.36,26.98,83.10,4.02,44.00,17.70,34.77
14,21,0.0,1,50.0,1816,1840,1475.571429,1671.233333,50.0,29,...,2.95,95.65,21.15,93.48,38.16,92.61,6.31,52.28,22.59,40.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,29,4.0,1,0.0,1597,1489,1630.900000,1435.363636,1.3,29,...,4.30,97.74,8.41,83.08,25.65,85.25,5.73,41.24,21.07,34.07
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,12.03,93.91,10.67,89.22,33.38,89.97,0.63,66.67,14.22,7.15
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,1.79,93.22,12.53,95.87,24.67,86.31,4.08,57.46,21.25,35.49
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,3.80,100.00,18.87,93.96,32.49,92.40,5.32,52.38,22.80,38.82


['duels90',
 'duelswonpct',
 'succssdefact90',
 'defduels90',
 'defduelswpct',
 'aerduels90',
 'aerduelspct',
 'sltackles90',
 'padjsltackles',
 'shotblocks90',
 'intrcpt90',
 'padjintrcpt',
 'fouls90',
 'ycards90',
 'rcards90',
 'passes90',
 'accpassespct',
 'fwdpasses90',
 'accfwdpassespct',
 'bpasses90',
 'accbpassespct',
 'latpasses90',
 'acclatpassespct',
 'shortmedpasses90',
 'accshortmedpassespct',
 'longpasses90',
 'acclongpassesct',
 'avgpasslen',
 'avglongpasslen']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'iterations': 1099, 'learning_rate': 0.0652461699054724, 'depth': 4, 'subsample': 0.9748195402772009, 'colsample_bylevel': 0.9421009537898084, 'min_data_in_leaf': 31}
thresh = 10
mae: 2.2277503864367585
mape: 0.5350268050316422
mae_below_thresh: 1.2283560832696596
mape_above_thresh: 0.26420031351146045


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1828,Axel Disasi,25,23/24,France,FR1,AS Monaco,England,GB1,Chelsea FC,1,30.00,42.0,45.00,42.0,41.957970,0.042030,0.001001
269,Arnaud Souquet,27,19/20,Belgium,BE1,KAA Gent,France,FR1,Montpellier HSC,1,3.50,4.0,2.80,4.0,3.942606,0.057394,0.014348
707,Antonio Barragán,33,20/21,Spain,ES1,Real Betis Balompié,Spain,ES1,Elche CF,1,1.60,0.8,0.00,0.8,0.742099,0.057901,0.072377
1048,Dimitri Foulquier,28,21/22,Spain,ES1,Granada CF,Spain,ES1,Valencia CF,1,3.50,4.0,2.50,4.0,4.069183,0.069183,0.017296
1666,Kévin Rodrigues,28,22/23,Spain,ES1,Real Sociedad,Türkiye,TR1,Adana Demirspor,1,1.80,2.3,0.00,2.3,2.228795,0.071205,0.030959
1756,Samet Akaydin,28,22/23,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,Fenerbahce,0,2.00,4.2,3.70,4.2,4.119551,0.080449,0.019155
1633,Jure Balkovec,27,22/23,Türkiye,TR1,Fatih Karagümrük,Türkiye,TR1,Alanyaspor,1,2.30,2.2,0.00,2.2,2.116047,0.083953,0.038160
260,Neven Subotic,30,19/20,France,FR1,AS Saint-Étienne,Germany,L1,1.FC Union Berlin,1,4.00,2.0,0.00,2.0,2.126404,0.126404,0.063202
271,Igor Diveev,19,19/20,Russia,RU1,FK Ufa,Russia,RU1,CSKA Moscow,1,3.50,4.5,1.50,4.5,4.371336,0.128664,0.028592
173,Alberto Moreno,27,19/20,England,GB1,Liverpool FC,Spain,ES1,Villarreal CF,1,9.00,6.0,0.00,6.0,5.870922,0.129078,0.021513


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1205,Reinildo Mandava,28,21/22,France,FR1,LOSC Lille,Spain,ES1,Atlético de Madrid,0,9.0,22.0,3.00,22.0,6.703854,15.296146,0.695279
875,Fikayo Tomori,23,21/22,England,GB1,Chelsea FC,Italy,IT1,AC Milan,1,28.0,50.0,34.40,50.0,36.803820,13.196180,0.263924
1317,Robin Gosens,27,22/23,Italy,IT1,Atalanta BC,Italy,IT1,Inter Milan,1,28.0,15.0,27.40,15.0,27.784784,12.784784,0.852319
1341,Pervis Estupiñán,24,22/23,Spain,ES1,Villarreal CF,England,GB1,Brighton & Hove Albion,1,20.0,32.0,17.80,32.0,20.994841,11.005159,0.343911
290,Sofyan Amrabat,23,19/20,Belgium,BE1,Club Brugge KV,Italy,IT1,Hellas Verona,0,3.0,14.5,3.65,14.5,3.742951,10.757049,0.741865
8,João Cancelo,25,19/20,Italy,IT1,Juventus FC,England,GB1,Manchester City,1,55.0,36.0,65.00,36.0,45.470250,9.470250,0.263063
485,Thomas Meunier,28,20/21,France,FR1,Paris Saint-Germain,Germany,L1,Borussia Dortmund,1,24.0,12.0,0.00,12.0,20.661832,8.661832,0.721819
139,Denis Suárez,25,19/20,Spain,ES1,FC Barcelona,Spain,ES1,Celta de Vigo,1,12.0,9.5,12.90,9.5,18.131157,8.631157,0.908543
178,Ramy Bensebaini,24,19/20,France,FR1,Stade Rennais FC,Germany,L1,Borussia Mönchengladbach,1,8.0,15.0,8.00,15.0,7.447035,7.552965,0.503531
1310,Niklas Süle,26,22/23,Germany,L1,Bayern Munich,Germany,L1,Borussia Dortmund,1,35.0,35.0,0.00,35.0,27.548121,7.451879,0.212911


In [8]:
scores = pd.read_csv('catboost_def.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(10).to_dict()

In [9]:
for _, features in feature_sets.items():
    pipeline(pos='DEF', features=features, full_feature_set=True)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,minsplayed,foot,height,weight,avgpasslen,dribbles90,acclongpassesct,accprpassespct
4,19,0.0,1,85.5,1816,1871,1429.238095,1561.400000,75.0,33,3128,1,189,89,21.02,0.78,57.58,80.16
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,1175,1,184,79,19.57,1.53,49.09,79.12
8,25,0.0,1,65.0,1871,2041,1561.400000,1696.800000,55.0,25,2096,1,182,74,19.02,6.83,62.50,78.23
12,21,0.0,1,55.0,1747,1842,1696.800000,1696.800000,35.0,35,3356,1,183,72,17.70,4.40,44.00,66.67
14,21,0.0,1,50.0,1816,1840,1475.571429,1671.233333,50.0,29,2809,1,186,78,22.59,1.51,52.28,77.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,29,4.0,1,0.0,1597,1489,1630.900000,1435.363636,1.3,29,2782,1,182,73,21.07,1.20,41.24,62.88
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,860,1,170,60,14.22,6.38,66.67,72.50
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,2959,1,191,84,21.25,1.06,57.46,62.63
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,1421,1,181,78,22.80,0.76,52.38,69.15


['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'avgpasslen',
 'dribbles90',
 'acclongpassesct',
 'accprpassespct']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'iterations': 4767, 'learning_rate': 0.04469485468419223, 'depth': 7, 'subsample': 0.5589843964169647, 'colsample_bylevel': 0.6351724860942549, 'min_data_in_leaf': 45}
thresh = 10
mae: 2.0906514434894885
mape: 0.46540463566736423
mae_below_thresh: 1.1477937521250465
mape_above_thresh: 0.24840267579454045


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
344,Fedor Kudryashov,32,19/20,Türkiye,TR1,Basaksehir FK,Russia,RU1,FC Sochi,1,1.75,1.2,0.0,1.2,1.194944,0.005056,0.004213
650,Diogo Queirós,21,20/21,Portugal,PO1,FC Porto,Portugal,PO1,FC Famalicão,1,3.20,3.0,0.0,3.0,3.019106,0.019106,0.006369
278,Reinildo Mandava,25,19/20,Portugal,PO1,B SAD,France,FR1,LOSC Lille,1,0.60,2.4,3.0,2.4,2.425866,0.025866,0.010777
288,Sergey Parshivlyuk,30,19/20,Russia,RU1,FK Rostov,Russia,RU1,Dinamo Moscow,1,3.00,1.8,0.0,1.8,1.827333,0.027333,0.015185
1048,Dimitri Foulquier,28,21/22,Spain,ES1,Granada CF,Spain,ES1,Valencia CF,1,3.50,4.0,2.5,4.0,3.972240,0.027760,0.006940
197,Christian Luyindama,25,19/20,Belgium,BE1,Standard Liège,Türkiye,TR1,Galatasaray,1,7.50,5.5,5.0,5.5,5.536447,0.036447,0.006627
831,Diogo Figueiras,29,20/21,Portugal,PO1,SC Braga,Portugal,PO1,FC Famalicão,0,0.65,0.8,0.0,0.8,0.857560,0.057560,0.071949
1656,Federico Ricca,27,22/23,Belgium,BE1,Club Brugge KV,Belgium,BE1,Oud-Heverlee Leuven,1,2.00,1.5,0.0,1.5,1.558376,0.058376,0.038918
2107,Jacob Rasmussen,26,23/24,Italy,IT1,ACF Fiorentina,Denmark,DK1,Bröndby IF,1,3.00,3.0,3.0,3.0,3.083312,0.083312,0.027771
2096,Portu,31,23/24,Spain,ES1,Real Sociedad,Spain,ES1,Getafe CF,1,3.50,3.5,3.5,3.5,3.586856,0.086856,0.024816


...

['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'avgpasslen',
 'dribbles90',
 'acclongpassesct',
 'accprpassespct',
 'shortmedpasses90',
 'passes90',
 'fouls90',
 'xg90',
 'rcvdlongpasses90']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'iterations': 724, 'learning_rate': 0.037352919907863304, 'depth': 7, 'subsample': 0.825780490052041, 'colsample_bylevel': 0.8974483005131563, 'min_data_in_leaf': 10}
thresh = 10
mae: 2.1949125377698047
mape: 0.5264743334908132
mae_below_thresh: 1.2233879032362711
mape_above_thresh: 0.2522529546013132


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
260,Neven Subotic,30,19/20,France,FR1,AS Saint-Étienne,Germany,L1,1.FC Union Berlin,1,4.00,2.000,0.0,2.000,2.040381,0.040381,0.020191
639,Filip Novak,30,20/21,Türkiye,TR1,Trabzonspor,Türkiye,TR1,Fenerbahce,1,2.80,2.000,0.0,2.000,2.071409,0.071409,0.035704
344,Fedor Kudryashov,32,19/20,Türkiye,TR1,Basaksehir FK,Russia,RU1,FC Sochi,1,1.75,1.200,0.0,1.200,1.093553,0.106447,0.088705
1651,Eli Dasa,29,22/23,Netherlands,NL1,Vitesse Arnhem,Russia,RU1,Dynamo Moscow,1,2.00,1.800,0.0,1.800,1.684929,0.115071,0.063929
841,Ahmet Oğuz,27,20/21,Türkiye,TR1,Kasimpasa,Türkiye,TR1,Sivasspor,0,0.45,0.800,0.0,0.800,0.917065,0.117065,0.146332
966,Aïssa Mandi,29,21/22,Spain,ES1,Real Betis Balompié,Spain,ES1,Villarreal CF,1,8.00,6.000,0.0,6.000,5.837237,0.162763,0.027127
1250,Gilles Dewaele,25,21/22,Belgium,BE1,KV Kortrijk,Belgium,BE1,Standard Liège,0,1.30,1.400,0.0,1.400,1.570961,0.170961,0.122115
2096,Portu,31,23/24,Spain,ES1,Real Sociedad,Spain,ES1,Getafe CF,1,3.50,3.500,3.5,3.500,3.328916,0.171084,0.048881
435,Bruno González,29,19/20,Spain,ES1,Getafe CF,Spain,ES1,Levante UD,0,2.00,1.200,0.0,1.200,1.382079,0.182079,0.151732
446,Mica Pinto,26,19/20,Netherlands,NL1,Fortuna Sittard,Netherlands,NL1,Sparta Rotterdam,0,0.65,0.525,0.0,0.525,0.730675,0.205675,0.391763


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
875,Fikayo Tomori,23,21/22,England,GB1,Chelsea FC,Italy,IT1,AC Milan,1,28.0,50.0,34.40,50.0,33.257086,16.742914,0.334858
1205,Reinildo Mandava,28,21/22,France,FR1,LOSC Lille,Spain,ES1,Atlético de Madrid,0,9.0,22.0,3.00,22.0,5.819879,16.180121,0.735460
1341,Pervis Estupiñán,24,22/23,Spain,ES1,Villarreal CF,England,GB1,Brighton & Hove Albion,1,20.0,32.0,17.80,32.0,19.337921,12.662079,0.395690
1833,Pedro Porro,23,23/24,Portugal,PO1,Sporting CP,England,GB1,Tottenham Hotspur,1,40.0,45.0,40.00,45.0,33.851617,11.148383,0.247742
290,Sofyan Amrabat,23,19/20,Belgium,BE1,Club Brugge KV,Italy,IT1,Hellas Verona,0,3.0,14.5,3.65,14.5,4.344243,10.155757,0.700397
562,Alexis Saelemaekers,21,20/21,Belgium,BE1,RSC Anderlecht,Italy,IT1,AC Milan,1,6.3,18.0,7.65,18.0,7.967436,10.032564,0.557365
1317,Robin Gosens,27,22/23,Italy,IT1,Atalanta BC,Italy,IT1,Inter Milan,1,28.0,15.0,27.40,15.0,24.064723,9.064723,0.604315
1744,Julian Ryerson,25,22/23,Germany,L1,1.FC Union Berlin,Germany,L1,Borussia Dortmund,0,4.0,13.0,5.00,13.0,4.174455,8.825545,0.678888
485,Thomas Meunier,28,20/21,France,FR1,Paris Saint-Germain,Germany,L1,Borussia Dortmund,1,24.0,12.0,0.00,12.0,20.823461,8.823461,0.735288
1828,Axel Disasi,25,23/24,France,FR1,AS Monaco,England,GB1,Chelsea FC,1,30.0,42.0,45.00,42.0,33.791404,8.208596,0.195443


In [10]:
with pd.option_context('display.max_colwidth', None):
    display(model_scores.sort_values('mape_above_thresh'))

,mae,mape,mae_below_thresh,mape_above_thresh,features
0,2.243049,0.581261,1.310031,0.245958,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90, xg90, rcvdlongpasses90, crosses6yard90, hdrgoals90, crossesright90]"
0,2.179351,0.495790,1.220007,0.246925,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90]"
0,2.090651,0.465405,1.147794,0.248403,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct]"
0,2.384477,0.593498,1.476125,0.251553,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90, xg90, rcvdlongpasses90, crosses6yard90, hdrgoals90, crossesright90, secassist90, goals90, bpasses90, accfinal3rdpassespct, rcvdpasses90, aerduelspct, pencnvrspct, avglongpasslen, directfk90, pens]"
0,2.194913,0.526474,1.223388,0.252253,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90, xg90, rcvdlongpasses90]"
0,2.126747,0.469666,1.149152,0.252573,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90]"
0,2.235227,0.520641,1.293611,0.258968,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90, xg90]"
0,2.262455,0.528494,1.272625,0.259305,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90, xg90, rcvdlongpasses90, crosses6yard90, hdrgoals90, crossesright90, secassist90, goals90, bpasses90, accfinal3rdpassespct, rcvdpasses90, aerduelspct, pencnvrspct, avglongpasslen, directfk90]"
0,2.238078,0.493623,1.241136,0.261416,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, avgpasslen, dribbles90, acclongpassesct, accprpassespct, shortmedpasses90, passes90, fouls90, xg90, rcvdlongpasses90, crosses6yard90, hdrgoals90, crossesright90, secassist90, goals90]"
0,2.227750,0.535027,1.228356,0.264200,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, duels90, duelswonpct, succssdefact90, defduels90, defduelswpct, aerduels90, aerduelspct, sltackles90, padjsltackles, shotblocks90, intrcpt90, padjintrcpt, fouls90, ycards90, rcards90, passes90, accpassespct, fwdpasses90, accfwdpassespct, bpasses90, accbpassespct, latpasses90, acclatpassespct, shortmedpasses90, accshortmedpassespct, longpasses90, acclongpassesct, avgpasslen, avglongpasslen]"
